In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import time
import mdtraj as md
import numpy as np

In [ ]:
prot = md.load_pdb('/home/eva/Documents/structures/structures-obc2-ff96/tet_L24E/GLU/6q5i_GLU_cv.pdb')
topology = prot.topology

In [ ]:
g1 = [atom.index for atom in topology.chain(0).atoms]
g2 = [atom.index for atom in topology.chain(1).atoms]
g3 = [atom.index for atom in topology.chain(2).atoms]
g4 = [atom.index for atom in topology.chain(3).atoms]

prmtop = AmberPrmtopFile('/home/eva/Documents/structures/structures-obc2-ff96/tet_L24E/GLU/6q5i_GLU.prmtop')
inpcrd = AmberInpcrdFile('/home/eva/Documents/structures/structures-obc2-ff96/tet_L24E/GLU/6q5i_GLU.inpcrd')
pdb = PDBFile('/home/eva/Documents/structures/structures-obc2-ff96/tet_L24E/GLU/6q5i_GLU_cv.pdb')

system = prmtop.createSystem(nonbondedMethod=CutoffNonPeriodic, nonbondedCutoff=1*nanometer,
        constraints=HBonds, implicitSolvent=OBC2)

expr = 'distance(g1,g2) + distance(g1,g3) + distance(g1,g4) + distance(g2,g3)+ distance(g2,g4) +distance(g3,g4)'

force = openmm.CustomCentroidBondForce(4, expr)
force.addGroup(g1)
force.addGroup(g2)
force.addGroup(g3)
force.addGroup(g4)
ainds=[0,1,2,3]
force.addBond(ainds)


cvforce = openmm.CustomCVForce('r')
cvforce.addCollectiveVariable('r', force) 
system.addForce(cvforce)

integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/picosecond, 0.002*picoseconds)

platform = Platform.getPlatformByName('CUDA')
properties = {'Precision': 'mixed'}

simulation = Simulation(prmtop.topology, system, integrator, platform, properties)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('/home/eva/Documents/gbsa-sims/cv-1/tet_L24E/GLU/output.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(50000000)

print(cvforce.getCollectiveVariableValues(simulation.context))